In [1]:
from pathlib import Path
import xarray as xr
import numpy as np
import pandas as pd

import importlib
import sys

In [2]:
from dask.distributed import Client

In [3]:
client = Client(
    n_workers=4, threads_per_worker=8, memory_limit=300e9,
    ip="0.0.0.0"
)
client

Client Scheduler: tcp://10.0.4.71:41267 Dashboard: http://10.0.4.71:8787/status,Cluster Workers: 4 Cores: 32 Memory: 1.20 TB


In [4]:
# parameters
project_path = Path.cwd() / '..' / '..' 
project_path = project_path.resolve()

data_path = "/gxfs_work1/geomar/smomw122/2022-02-09_spg_fresh_blob_202104_raw_data/"

interim_data_path = Path('data/interim/endtracks/plusDist/')

sectionPath = Path('data/external/')
sectionFilename = 'osnap_pos_wp.txt'
sectionname = 'osnap'

year = 1990

In [5]:
year_str = str(year)

In [6]:
interim_data_filename = "endtracks_20211215_randomvel_mxl_osnap_backwards_"+year_str + ".nc"

In [7]:
# data_stores_subsets = list(sorted(Path(data_path).glob("*_????_subset.zarr/")))[:use_number_subset_years]
data_stores_subsets = list(sorted(Path(data_path).glob(f"*{year:04d}.zarr/")))

In [8]:
display(data_stores_subsets)

[PosixPath('/gxfs_work1/geomar/smomw122/2022-02-09_spg_fresh_blob_202104_raw_data/tracks_randomvel_mxl_osnap_backwards_1990.zarr')]

In [9]:
ds_subsets = xr.concat(
    [
        xr.open_zarr(store, decode_cf=False)
        for store in data_stores_subsets
    ],
    dim="traj",
)

display(ds_subsets)
print(ds_subsets.nbytes / 1e9, "GiB")

<xarray.Dataset>
Dimensions:  (traj: 5164156, obs: 731)
Dimensions without coordinates: traj, obs
Data variables:
    lat      (traj, obs) float32 dask.array<chunksize=(60000, 731), meta=np.ndarray>
    lon      (traj, obs) float32 dask.array<chunksize=(60000, 731), meta=np.ndarray>
    mxl      (traj, obs) float32 dask.array<chunksize=(60000, 731), meta=np.ndarray>
    salt     (traj, obs) float32 dask.array<chunksize=(60000, 731), meta=np.ndarray>
    temp     (traj, obs) float32 dask.array<chunksize=(60000, 731), meta=np.ndarray>
    time     (traj, obs) float64 dask.array<chunksize=(60000, 731), meta=np.ndarray>
    uvel     (traj, obs) float32 dask.array<chunksize=(60000, 731), meta=np.ndarray>
    vvel     (traj, obs) float32 dask.array<chunksize=(60000, 731), meta=np.ndarray>
    z        (traj, obs) float32 dask.array<chunksize=(60000, 731), meta=np.ndarray>
Attributes:
    Conventions:            CF-1.6/CF-1.7
    feature_type:           trajectory
    ncei_template_version:  NCEI_NetCDF_Trajectory_Template_v2.0
    parcels_mesh:           spherical
    parcels_version:        2.2.2

150.99992144 GiB


## add cumulative track lengths to file

In [10]:
ds_subsets_lat_diff=ds_subsets.lat.diff(dim='obs', n=1)
ds_subsets_lon_diff=ds_subsets.lat.diff(dim='obs', n=1)


In [11]:
# Calculate Great Circle distance between requeted point and 
# matched grid point. 
# Based on https://andrew.hedges.name/experiments/haversine/

# copied from internet

# approximate radius of earth in km
R = 6373000.0

lat1 = xr.ufuncs.radians(ds_subsets.lat)
lon1 = xr.ufuncs.radians(ds_subsets.lon)
lat2 = lat1.shift(obs=1)
lon2 = lon1.shift(obs=1)

dlat = lat2 - lat1
dlon = lon2 - lon1

a = np.sin(dlat / 2)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon / 2)**2
c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1 - a))

distance = R * c


In [12]:
ds_subsets = ds_subsets.assign({'dist':distance.cumsum(dim='obs')})

In [13]:
ds_subsets.dist.attrs = {
    'long_name': 'alongtrack distance from OSNAP',
    'units': 'metres'
}

In [14]:
ds_subsets

<xarray.Dataset>
Dimensions:  (traj: 5164156, obs: 731)
Dimensions without coordinates: traj, obs
Data variables:
    lat      (traj, obs) float32 dask.array<chunksize=(60000, 731), meta=np.ndarray>
    lon      (traj, obs) float32 dask.array<chunksize=(60000, 731), meta=np.ndarray>
    mxl      (traj, obs) float32 dask.array<chunksize=(60000, 731), meta=np.ndarray>
    salt     (traj, obs) float32 dask.array<chunksize=(60000, 731), meta=np.ndarray>
    temp     (traj, obs) float32 dask.array<chunksize=(60000, 731), meta=np.ndarray>
    time     (traj, obs) float64 dask.array<chunksize=(60000, 731), meta=np.ndarray>
    uvel     (traj, obs) float32 dask.array<chunksize=(60000, 731), meta=np.ndarray>
    vvel     (traj, obs) float32 dask.array<chunksize=(60000, 731), meta=np.ndarray>
    z        (traj, obs) float32 dask.array<chunksize=(60000, 731), meta=np.ndarray>
    dist     (traj, obs) float32 dask.array<chunksize=(60000, 731), meta=np.ndarray>
Attributes:
    Conventions:            CF-1.6/CF-1.7
    feature_type:           trajectory
    ncei_template_version:  NCEI_NetCDF_Trajectory_Template_v2.0
    parcels_mesh:           spherical
    parcels_version:        2.2.2

## Extract data on osnap line (initialised positions)

In [15]:
ds_subsets_osnap = ds_subsets.isel(obs=0)

## Update some attributes

In [16]:
# ds_subsets_osnap['vol_trans_normal'] = 
ds_subsets_osnap.mxl.attrs = {'units':'m','long_name':'mixed layer depth'}
ds_subsets_osnap.salt.attrs = {'units':'PSU','long_name':'salinity'}
ds_subsets_osnap.temp.attrs = {'units':'degC','long_name':'temperature'}
ds_subsets_osnap.uvel.attrs = {'units':'degrees_east/second','long_name':'u velocity (raw)'}
ds_subsets_osnap.vvel.attrs = {'units':'degrees_north/second','long_name':'v velocity (raw)'}

## Flag tracks by source region and pathway

In [17]:
def apply_left_of_line(ds, lon_1, lon_2, lat_1, lat_2):
    '''Apply an area crossing criterion.
    
    Larvae in ds selected while they are in a selected area.
    '''
    # particles are selected if they pass through given area.
    position =  ((lon_2 -lon_1) * (ds.lat - lat_1) - 
                     (ds.lon - lon_1) * (lat_2 - lat_1))
                        
    return position > 0.0, position < 0

#### from Labrador sea or from Gulf Stream

In [18]:
# from labrador sea
ds_in1, ds_notin1 = apply_left_of_line(ds_subsets,-75,-40,40,65)
ds_in2, ds_notin2 = apply_left_of_line(ds_subsets,-100,-58.2,48,52)
ds_in3, ds_notin3 = apply_left_of_line(ds_subsets,-45,-45,60,70)
ds_lab_in = ds_in1*ds_in2*ds_in3
# from west of 60W, south of Flemish Cap (to test path from labrador sea)
ds_in1, ds_notin1 = apply_left_of_line(ds_subsets,-60,-60,33,63)
ds_in2, ds_notin2 = apply_left_of_line(ds_subsets,-58.2,-100,52,48)
ds_60w_in = ds_in1*ds_in2
# from gulf stream
ds_in1, ds_notin1 = apply_left_of_line(ds_subsets,-60,-100,33,33)
ds_in2, ds_notin2 = apply_left_of_line(ds_subsets,-44,-44,0,33)
ds_gst_in = ds_in1 * ds_in2


In [19]:
# check trajectory routes
LabCu = ds_lab_in.max("obs")
LC60W = ds_60w_in.max("obs")
GulfS = ds_gst_in.max("obs")

# check when lef lab sea,crossed 60w or gulf stream. defaults to zero
LabCu_exit_index = ds_lab_in.argmax(axis=1)
LC60W_exit_index = ds_60w_in.argmax(axis=1)
GulfS_exit_index = ds_gst_in.argmax(axis=1)

LabCu_exit_index = LabCu_exit_index.where(LabCu_exit_index > 0,len(ds_subsets.obs)-1)
LC60W_exit_index = LC60W_exit_index.where(LC60W_exit_index > 0,len(ds_subsets.obs)-1)
GulfS_exit_index = GulfS_exit_index.where(GulfS_exit_index > 0,len(ds_subsets.obs)-1)

In [20]:
# check if most recently left Lab Sea
LabCu_is_source = (LabCu_exit_index < GulfS_exit_index)
GulfS_is_source = (LabCu_exit_index > GulfS_exit_index)
LC60W_is_path = (LabCu_exit_index > LC60W_exit_index).where(LabCu_is_source,False)

In [21]:
LCdir_is_path = LabCu_is_source.where(LC60W_is_path == False, False)
other_is_source = (LabCu_is_source == False).where(GulfS_is_source == False, False)

flag particles on osnap line by origin

In [22]:
ds_subsets_osnap = ds_subsets_osnap.assign({'LabCu_is_source':LabCu_is_source})
ds_subsets_osnap = ds_subsets_osnap.assign({'LC60W_is_path':LC60W_is_path})
ds_subsets_osnap = ds_subsets_osnap.assign({'LCdir_is_path':LCdir_is_path})
ds_subsets_osnap = ds_subsets_osnap.assign({'GulfS_is_source':GulfS_is_source})
ds_subsets_osnap = ds_subsets_osnap.assign({'other_is_source':other_is_source})


flag by pathway for Labrador Current parcels

In [23]:
ds_subsets_osnap.LabCu_is_source.attrs = {'long_name':'flag from Labrador Current'}
ds_subsets_osnap.LC60W_is_path.attrs   = {'long_name':'flag from LC via 60W'}
ds_subsets_osnap.LCdir_is_path.attrs   = {'long_name':'flag from LC direct'}
ds_subsets_osnap.GulfS_is_source.attrs = {'long_name':'flag from Gulf Stream'}
ds_subsets_osnap.other_is_source.attrs = {'long_name':'flag source not found'}

## Find the 'obs' index of the point where parcel leaves the source region

### known source regions

In [24]:
# test individaul positions to see when the source was left
# defaults to zero if particular source was not on track
LabCu_exit_index = (ds_lab_in.where(ds_subsets_osnap.LabCu_is_source,False)).argmax(axis=1)
GulfS_exit_index = (ds_gst_in.where(ds_subsets_osnap.GulfS_is_source,False)).argmax(axis=1)

# combine in to one array. Nonzero numbers should not overlap for 
# Lab current and Gulf Stream
exit_index = LabCu_exit_index + GulfS_exit_index
# convert zeros to max dim obs
exit_index = exit_index.where(exit_index > 0, len(ds_subsets.obs)-1)

### index last non nan value for 'other' parcels

In [25]:
a = ds_subsets.lat  # just a random selection of variable, nans the same for all variables
b = (~np.isnan(a)).cumsum(dim='obs').argmax(dim='obs') # finds last non-nan in dim 'obs'. nicked from stackoverflow search
exit_index = xr.ufuncs.minimum(exit_index, b).compute()

In [26]:
display(exit_index)

<xarray.DataArray (traj: 5164156)>
array([244, 231, 518, ..., 246, 217, 240])
Dimensions without coordinates: traj

### extract source positions from full array

We need to take away the _HUGE_ lookup of obs steps from the frontend process.
So we create a 2d data var (lazily with dask) filled with obs and then do a `.where()` and a reduction.

In [27]:
ds_subsets.coords["obs"] = np.arange(ds_subsets.dims["obs"])
ds_subsets["obs2d"] = (
    xr.full_like(ds_subsets.lat, 0, dtype="int")
    + ds_subsets.coords["obs"]
)

In [28]:
%%time

ds_subsets_sourc = ds_subsets.where(
    ds_subsets.obs2d == exit_index
).mean("obs")

ds_subsets_sourc = ds_subsets_sourc.compute()

display(ds_subsets_sourc)

<xarray.Dataset>
Dimensions:  (traj: 5164156)
Dimensions without coordinates: traj
Data variables:
    lat      (traj) float32 31.63 32.82 55.45 69.31 ... 32.64 32.61 54.77 30.76
    lon      (traj) float32 -78.98 -77.03 -54.99 -20.68 ... -77.08 -54.71 -79.61
    mxl      (traj) float32 22.45 47.42 371.2 24.18 ... 46.75 14.61 24.83 21.39
    salt     (traj) float32 35.1 35.59 34.95 32.9 ... 36.14 35.9 33.63 36.37
    temp     (traj) float32 9.492 12.87 3.78 -1.532 ... 16.81 15.16 -0.508 18.5
    time     (traj) float64 2.102e+08 2.158e+08 9.184e+07 ... 2.53e+08 2.43e+08
    uvel     (traj) float32 6.725e-06 4.428e-06 ... 2.723e-06 5.692e-06
    vvel     (traj) float32 5.376e-06 3.197e-06 ... -1.239e-06 1.084e-05
    z        (traj) float32 253.4 420.7 742.8 30.93 ... 345.3 382.4 93.01 204.5
    dist     (traj) float32 1.687e+07 1.87e+07 2.219e+07 ... 1.43e+07 1.383e+07
    obs2d    (traj) float64 244.0 231.0 518.0 20.0 ... 347.0 246.0 217.0 240.0

CPU times: user 8.96 s, sys: 1.23 s, total: 10.2 s
Wall time: 47 s


add the source and pathway flags to ds_subsets_sourc to match for xr.concat

In [29]:
# flag particles on osnap line by origin and pathway
ds_subsets_sourc = ds_subsets_sourc.assign({'LabCu_is_source':LabCu_is_source})
ds_subsets_sourc = ds_subsets_sourc.assign({'LC60W_is_path':LC60W_is_path})
ds_subsets_sourc = ds_subsets_sourc.assign({'LCdir_is_path':LCdir_is_path})
ds_subsets_sourc = ds_subsets_sourc.assign({'GulfS_is_source':GulfS_is_source})
ds_subsets_sourc = ds_subsets_sourc.assign({'other_is_source':other_is_source})


In [30]:
ds_subsets_sourc.LabCu_is_source.attrs = {'long_name':'flag from Labrador Current'}
ds_subsets_sourc.LC60W_is_path.attrs   = {'long_name':'flag from LC via 60W'}
ds_subsets_sourc.LCdir_is_path.attrs   = {'long_name':'flag from LC direct'}
ds_subsets_sourc.GulfS_is_source.attrs = {'long_name':'flag from Gulf Stream'}
ds_subsets_sourc.other_is_source.attrs = {'long_name':'flag source not found'}

### combine source and osnap positions and characteristics

In [31]:
ds_subsets_paths = xr.concat([ds_subsets_osnap,
                             ds_subsets_sourc],
                             dim='ends')

### flag particles entering from north from Greenland Sea or Davis Strait

In [32]:
b = b.compute()

In [33]:
display(b)

<xarray.DataArray 'lat' (traj: 5164156)>
array([300, 334, 730, ..., 259, 309, 416])
Dimensions without coordinates: traj

In [34]:
%%time

ds_subsets_domexi = ds_subsets.where(
    ds_subsets.obs2d == b
).mean("obs")

ds_subsets_domexi = ds_subsets_domexi.compute()

display(ds_subsets_domexi)

# ds_subsets_domexi = ds_subsets.isel(
#     traj=xr.DataArray(range(len(ds_subsets.traj)),dims='traj'),
#     obs=b
# )


<xarray.Dataset>
Dimensions:  (traj: 5164156)
Dimensions without coordinates: traj
Data variables:
    lat      (traj) float32 20.01 20.01 36.2 69.31 ... 20.01 20.01 66.45 20.01
    lon      (traj) float32 -85.82 -86.86 -64.45 -20.68 ... -86.57 -60.66 -86.48
    mxl      (traj) float32 24.2 24.49 78.64 24.18 ... 16.65 19.39 18.79 16.6
    salt     (traj) float32 34.98 34.92 35.52 32.9 ... 36.22 35.84 31.56 36.3
    temp     (traj) float32 9.526 8.763 12.03 -1.532 ... 14.94 0.009543 17.84
    time     (traj) float64 1.861e+08 1.715e+08 ... 2.133e+08 1.672e+08
    uvel     (traj) float32 -8.555e-07 3.785e-07 ... 6.335e-07 1.688e-06
    vvel     (traj) float32 1.48e-06 5.75e-07 1.767e-06 ... -1.73e-06 3.358e-06
    z        (traj) float32 504.9 432.4 453.3 30.93 ... 333.4 301.8 14.02 265.1
    dist     (traj) float32 2.142e+07 2.451e+07 ... 1.703e+07 3.264e+07
    obs2d    (traj) float64 300.0 334.0 730.0 20.0 ... 573.0 259.0 309.0 416.0

CPU times: user 9.26 s, sys: 1.25 s, total: 10.5 s
Wall time: 42.1 s


In [35]:
# counts how many times enters or leaves lab sea
spgnoloop = (
    abs(ds_lab_in.astype(int).diff(dim='obs')).sum(dim='obs') < 3
).compute()

In [36]:
# from Hudson Bay
ds_in1, ds_notin1 = apply_left_of_line(ds_subsets,-68,-68,33,63)
ds_in2, ds_notin2 = apply_left_of_line(ds_subsets,-95,-60,52,52)
ds_hud_in = ds_in1*ds_in2

HudBa = ds_hud_in.max("obs").compute()


In [37]:
# HudBa.data.compute()

In [38]:
Green_is_source = (ds_subsets_paths.isel(ends=0).LabCu_is_source &
                  (ds_subsets_domexi.lat > 65) & 
                  (ds_subsets_domexi.lon > -44) &
                  spgnoloop).compute()
Davis_is_source = (ds_subsets_paths.isel(ends=0).LabCu_is_source &
                  (ds_subsets_domexi.lat > 65) & 
                  (ds_subsets_domexi.lon < -44) &
                  spgnoloop).compute()
Hudba_is_source = (ds_subsets_paths.isel(ends=0).LabCu_is_source & 
                  HudBa &
                  spgnoloop).compute()

In [39]:
Green_is_source = xr.concat([(Green_is_source),(Green_is_source)],dim='ends').compute()
ds_subsets_paths = ds_subsets_paths.assign({'Green_is_source':Green_is_source})
Davis_is_source = xr.concat([(Davis_is_source),(Davis_is_source)],dim='ends').compute()
ds_subsets_paths = ds_subsets_paths.assign({'Davis_is_source':Davis_is_source})
Hudba_is_source = xr.concat([(Hudba_is_source),(Hudba_is_source)],dim='ends').compute()
ds_subsets_paths = ds_subsets_paths.assign({'Hudba_is_source':Hudba_is_source})


add the source and pathway flags to ds_subsets_sourc to match for xr.concat

In [40]:
ds_subsets_paths.Davis_is_source.attrs = {'long_name':'flag from Davis Strait'}
ds_subsets_paths.Green_is_source.attrs = {'long_name':'flag from Greenland Sea'}
ds_subsets_paths.Hudba_is_source.attrs = {'long_name':'flag from Hudson Bay'}


## We want to test for tracks which route north of osnap line between source and final times

This is because a common strategy is to remove these from analysis and only consider the 'direct' paths.

### Test particle positions

### section position data

In [41]:
lonlat = xr.Dataset(pd.read_csv(project_path / sectionPath / sectionFilename,delim_whitespace=True))

#### south/north of osnap-e

In [42]:
# do north and south separately because of missing values

south = xr.Dataset()
north = xr.Dataset()
epsilon = 0.05
for i in range(len(lonlat.lon)-1):
    south['subsect'+str(i)],north['subsect'+str(i)] = apply_left_of_line(ds_subsets,lonlat.lon[i+1],lonlat.lon[i],lonlat.lat[i+1]+epsilon,lonlat.lat[i]+epsilon)

# check in osnap east 
south_oe,north_oe = apply_left_of_line(ds_subsets,-44,-44,30,60)

# south_a = south.subsect0 + south.subsect1 + south.subsect2 
# south_b = south.subsect3 * south.subsect4 * south.subsect5
south_b = south.subsect4 * south.subsect5
south_c = south.subsect6 + south.subsect7 + south.subsect8 
south_d = south.subsect8 * south.subsect9 * south.subsect10 * south.subsect11 
# south_e = south.subsect12
# south_all = south_a * south_c * south_e * (south_b + south_d)
south_all = south_oe + (south_c * (south_b + south_d))

# north_a = north.subsect0 * north.subsect1 * north.subsect2 
# north_b = north.subsect3 + north.subsect4 + north.subsect5
north_b = north.subsect4 + north.subsect5
north_c = north.subsect6 * north.subsect7 * north.subsect8 
north_d = north.subsect8 + north.subsect9 + north.subsect10 + north.subsect11
# north_e = north.subsect12
# north_all = north_a + north_c + north_e + (north_b * north_d)
north_all = north_oe * (north_c  + (north_b * north_d))


In [43]:
lonlat

<xarray.Dataset>
Dimensions:  (dim_0: 13)
Coordinates:
  * dim_0    (dim_0) int64 0 1 2 3 4 5 6 7 8 9 10 11 12
Data variables:
    lat      (dim_0) float64 52.09 52.67 53.59 59.11 ... 57.99 57.47 57.1 56.73
    lon      (dim_0) float64 -56.16 -52.1 -49.78 -47.34 ... -12.71 -9.275 -5.77

In [44]:
north_all = north_all.reset_coords(drop=True)
south_all = south_all.reset_coords(drop=True)

In [45]:
north_all

<xarray.DataArray (traj: 5164156, obs: 731)>
dask.array<mul, shape=(5164156, 731), dtype=bool, chunksize=(60000, 731), chunktype=numpy.ndarray>
Coordinates:
  * obs      (obs) int64 0 1 2 3 4 5 6 7 8 ... 723 724 725 726 727 728 729 730
Dimensions without coordinates: traj

In [46]:
north_all = north_all.persist()

In [47]:
# test individual positions to see when the parcel was first (in 'obs', last in time) north of osnap line
# defaults to zero if particular source was not on track
north_osnap_index = (north_all).argmax(axis=1).compute()
# convert zeros to max dim obs
north_osnap_index = north_osnap_index.where(north_osnap_index > 0,len(ds_subsets.obs)-1).compute()

Check if found north of osnap e between leaving source and arriving at osnap and flag to ds_subsets_paths

In [48]:
north_osnap = xr.concat([(north_osnap_index < exit_index),(north_osnap_index < exit_index)],dim='ends')
ds_subsets_paths = ds_subsets_paths.assign({'north_of_osnap':north_osnap})
ds_subsets_paths.north_of_osnap.attrs = {'long_name':'flag path goes north of osnap-e'}
display(ds_subsets_paths)

<xarray.Dataset>
Dimensions:          (traj: 5164156, ends: 2)
Dimensions without coordinates: traj, ends
Data variables: (12/20)
    obs2d            (traj) float64 244.0 231.0 518.0 20.0 ... 246.0 217.0 240.0
    lat              (ends, traj) float32 dask.array<chunksize=(1, 60000), meta=np.ndarray>
    lon              (ends, traj) float32 dask.array<chunksize=(1, 60000), meta=np.ndarray>
    mxl              (ends, traj) float32 dask.array<chunksize=(1, 60000), meta=np.ndarray>
    salt             (ends, traj) float32 dask.array<chunksize=(1, 60000), meta=np.ndarray>
    temp             (ends, traj) float32 dask.array<chunksize=(1, 60000), meta=np.ndarray>
    ...               ...
    GulfS_is_source  (ends, traj) bool dask.array<chunksize=(1, 60000), meta=np.ndarray>
    other_is_source  (ends, traj) bool dask.array<chunksize=(1, 60000), meta=np.ndarray>
    Green_is_source  (ends, traj) bool False False False ... False False False
    Davis_is_source  (ends, traj) bool False False False ... False True False
    Hudba_is_source  (ends, traj) bool False False False ... False True False
    north_of_osnap   (ends, traj) bool True True True True ... False False False
Attributes:
    Conventions:            CF-1.6/CF-1.7
    feature_type:           trajectory
    ncei_template_version:  NCEI_NetCDF_Trajectory_Template_v2.0
    parcels_mesh:           spherical
    parcels_version:        2.2.2

## Output to netcdf

In [49]:
ds_subsets_paths = ds_subsets_paths.persist()

In [50]:
!mkdir -p {str((project_path / interim_data_path / interim_data_filename).parent)}

In [51]:
ds_subsets_paths.to_netcdf(project_path / interim_data_path / interim_data_filename)

In [52]:
conda list

# packages in environment at /opt/conda:
#
# Name                    Version                   Build  Channel
_libgcc_mutex             0.1                 conda_forge    conda-forge
_openmp_mutex             4.5                       1_gnu    conda-forge
alembic                   1.7.3              pyhd8ed1ab_0    conda-forge
ansiwrap                  0.8.4                      py_0    conda-forge
anyio                     3.3.2            py38h578d9bd_0    conda-forge
appdirs                   1.4.4              pyh9f0ad1d_0    conda-forge
argon2-cffi               20.1.0           py38h497a2fe_2    conda-forge
asciitree                 0.3.3                      py_2    conda-forge
async_generator           1.10                       py_0    conda-forge
attrs                     21.2.0             pyhd8ed1ab_0    conda-forge
babel                     2.9.1              pyh44b312d_0    conda-forge
backcall                  0.2.0              pyh9f0ad1d_0    conda-forge
backports     

/opt/conda/lib/python3.8/site-packages/distributed/client.py:4716: RuntimeWarning: coroutine 'wait_for' was never awaited
  c.close(timeout=3)
/opt/conda/lib/python3.8/site-packages/distributed/client.py:4716: RuntimeWarning: coroutine 'Client._close' was never awaited
  c.close(timeout=3)
/opt/conda/lib/python3.8/site-packages/distributed/client.py:4716: RuntimeWarning: coroutine 'wait_for' was never awaited
  c.close(timeout=3)
/opt/conda/lib/python3.8/site-packages/distributed/client.py:4716: RuntimeWarning: coroutine 'Client._close' was never awaited
  c.close(timeout=3)
/opt/conda/lib/python3.8/site-packages/distributed/client.py:4716: RuntimeWarning: coroutine 'wait_for' was never awaited
  c.close(timeout=3)
/opt/conda/lib/python3.8/site-packages/distributed/client.py:4716: RuntimeWarning: coroutine 'Client._close' was never awaited
  c.close(timeout=3)
/opt/conda/lib/python3.8/site-packages/distributed/client.py:4716: RuntimeWarning: coroutine 'wait_for' was never awaited
  c.c